In [2]:
import finvizfinance as fz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from finvizfinance.quote import finvizfinance, Statements, Quote
import yfinance as yf
import math


In [2]:
tickers = [
    "EQIX", "DLR",
    "PWR", "ETN", "ABB",
    "VRT", "TT", "JCI",
    "SMCI", "HPE", "DELL",
    "ANET", "AVGO", "CSCO",
    "MRVL", "COHR", "LITE", "INFN",
    "MU", "ON", "STM", "ADI", "TXN",
    "TSM", "INTC", "ASX", "AMKR", "PLAB",
    "SNPS", "CDNS", "ANSS",
    "STX", "WDC",
    "ASML", "AMAT", "KLAC", "LRCX",
    "NVDA",
    "GOOG", "ADBE",
    "000660.KS", "INDIGO.NS", "NVO", "UNH", "RHM.DE", "VOLTAMP.NS", "ADANIPOWER.NS", "WAAREEENER.BO"
]

# tickers = [
#     "NVO", "UNH", "000660.KS", "INDIGO.NS",
#     "TSM", 
#     "ASML",
#     "NVDA",
#     "GOOG", "GOOGL", "ADBE"
# ]

# tickers = ["TSM"]


In [3]:
# --- simple_buy_score.py (Yahoo Finance version) ------------------------------
# Minimal 2-year buy score using Yahoo's info_dict + quarterly dfs.
# Pillars: Growth (rev TTM), Profitability (GM/OM/ROE + FCF margin),
#          Valuation (Fwd P/E, PEG), Safety (D/E, Current Ratio).
# Uses only the rows/keys you said you have.

# ---------- helpers ----------
def _to_float(x, default=0.0):
    if x is None:
        return default
    try:
        return float(x)
    except Exception:
        s = str(x).replace(',', '').strip()
        if s.endswith('%'):
            s = s[:-1]
        try:
            return float(s)
        except Exception:
            return default

def _to_pct(x, default=0.0):
    """Return as percentage (0..100). Yahoo margins/ROE are typically decimals (e.g., 0.58)."""
    v = _to_float(x, default)
    if v is None:
        return default
    # If it looks like a fraction (<=1.5), treat as 0..1 and convert to %
    return v * 100.0 if -1.5 <= v <= 1.5 else v

def _pos(x, lo, hi):
    """Higher is better. Linear 0..1 between [lo, hi]."""
    if x <= lo: return 0.0
    if x >= hi: return 1.0
    return (x - lo) / (hi - lo)

def _neg(x, lo, hi):
    """Lower is better. Linear 0..1 between [lo, hi]."""
    if x <= lo: return 1.0
    if x >= hi: return 0.0
    return 1.0 - (x - lo) / (hi - lo)

def _series_recent4(df, row_label):
    """Return list of 4 most-recent values for a given row (columns most-recent first)."""
    if row_label not in df.index:
        return [0.0, 0.0, 0.0, 0.0]
    row = df.loc[row_label]
    n = min(4, row.shape[0])
    vals = [_to_float(row.iloc[i], 0.0) for i in range(n)]
    while len(vals) < 4:
        vals.append(vals[-1] if vals else 0.0)
    return vals

def _weighted_recent4(values, weights=(1.0, 0.75, 0.50, 0.25)):
    return sum(values[i]*weights[i] for i in range(4)) / sum(weights)

# ---------- metric extraction from dfs ----------
def extract_from_quarterlies(income_df, cashflow_df, balance_df, info_dict):
    # Revenue TTM growth (current 4q vs previous 4q) from income df
    if ('Total Revenue' in income_df.index) and (income_df.shape[1] >= 8):
        rev = income_df.loc['Total Revenue']
        rev_now  = sum(_to_float(rev.iloc[i])     for i in range(4))      # cols 0..3
        rev_prev = sum(_to_float(rev.iloc[i])     for i in range(4, 8))   # cols 4..7
        rev_ttm_growth = ((rev_now - rev_prev) / rev_prev * 100.0) if rev_prev > 0 else 0.0
    else:
        # fallback: Yahoo provides revenueGrowth as a decimal (e.g., 0.27)
        rev_ttm_growth = _to_pct(info_dict.get('revenueGrowth', 0.0))

    # FCF margin: weighted last 4 quarters (linear decay)
    fcf4 = _series_recent4(cashflow_df, 'Free Cash Flow')
    rev4 = _series_recent4(income_df,   'Total Revenue')
    fcf_w = _weighted_recent4(fcf4)
    rev_w = _weighted_recent4(rev4)
    fcf_margin = (fcf_w / rev_w * 100.0) if rev_w > 0 else 0.0

    # Current Ratio: compute from balance sheet if available, else fallback to info_dict['currentRatio']
    if ('Current Assets' in balance_df.index) and ('Current Liabilities' in balance_df.index):
        ca = _to_float(balance_df.loc['Current Assets'].iloc[0])
        cl = _to_float(balance_df.loc['Current Liabilities'].iloc[0])
        curr_ratio = (ca / cl) if (cl and cl != 0) else _to_float(info_dict.get('currentRatio', 1.0))
    else:
        curr_ratio = _to_float(info_dict.get('currentRatio', 1.0))

    # Debt/Equity: compute from balance sheet if possible, else fallback to info_dict['debtToEquity']
    if ('Total Debt' in balance_df.index) and ('Stockholders Equity' in balance_df.index):
        
        td = _to_float(balance_df.loc['Total Debt'].iloc[0], 0.0)
        se = _to_float(balance_df.loc['Stockholders Equity'].iloc[0], 0.0)
        debt_eq = (td / se) if (se > 0 and not math.isnan(td)) else _to_float(info_dict.get('debtToEquity', 0.0)/100)
        
    else:
        debt_eq = _to_float(info_dict.get('debtToEquity', 0.0)/100)

    return rev_ttm_growth, fcf_margin, debt_eq, curr_ratio

# ---------- final score ----------
def buy_score(info_dict, income_df, cashflow_df, balance_df):
    # Profitability (as %)
    gm  = _to_pct(info_dict.get('grossMargins', 0))      # %
    om  = _to_pct(info_dict.get('operatingMargins', 0))  # %
    roe = _to_pct(info_dict.get('returnOnEquity', 0))    # %

    # Valuation
    fpe = _to_float(info_dict.get('forwardPE', 0))                     # number
    peg = _to_float(info_dict.get('trailingPegRatio', 0))              # number

    # From dfs (with fallbacks)
    rev_g, fcf_margin, debt_eq, curr_ratio = extract_from_quarterlies(
        income_df, cashflow_df, balance_df, info_dict
    )

    # Subscores (0..1) — simple ranges for 2-year horizon
    growth = _pos(rev_g, 5, 40)  # 5%..40% revenue TTM growth

    profitability = (
        _pos(gm, 40, 70) +
        _pos(om, 15, 45) +
        _pos(roe, 10, 40) +
        _pos(fcf_margin, 5, 35)
    ) / 4.0

    # If PEG missing or <=0, weight valuation more on forwardPE
    if peg <= 0:
        valuation = _neg(fpe, 12, 40)
    else:
        valuation = (_neg(fpe, 12, 40) + _neg(peg, 1.0, 3.0)) / 2.0

    safety = (
        _neg(debt_eq, 0.0, 1.0) +
        _pos(curr_ratio, 1.0, 3.0)
    ) / 2.0

    score = (
        0.30*growth +
        0.30*profitability +
        0.25*valuation +
        0.15*safety
    ) * 100.0
    
    metric_vals = [gm, om, roe, fpe, peg, rev_g, fcf_margin, debt_eq, curr_ratio, growth, profitability, valuation, safety, score]

    return round(score, 1), metric_vals

def score_label(score):
    if score >= 75: return "BUY (High Conviction)"
    if score >= 60: return "ACCUMULATE / HOLD"
    return "AVOID / WATCHLIST"

# ---------- convenience ranker ----------
def rank_stocks(bundles):
    """
    bundles = list of dicts, each:
      {'fund': condensed information about the stock,
       'inc': income_df_q,
       'cf': cashflow_df_q,
       'bs': balance_df_q}
    Returns sorted list of (Name/Symbol, Score, Label).
    """
    out = []
    ticker_df = pd.DataFrame(columns=["Company", "Gross Margin", "Oper. Margin", "ROE", "Forward P/E", "PEG", "Revenue TTM Growth", "FCF Margin", "Debt/Equity", "Current Ratio", "Growth", "Profitability", "Valuation", "Safety", "Score"])

    for b in bundles:
        name = (
            b['fund'].get('longName')
            or b['fund'].get('shortName')
            or b['fund'].get('symbol')
            or 'Unknown'
        )
        sc, metric_vals = buy_score(b['fund'], b['inc'], b['cf'], b['bs'])
        ticker_df.loc[len(ticker_df)] = [name] + metric_vals
        out.append((name, sc, score_label(sc)))
    return sorted(out, key=lambda x: x[1], reverse=True), ticker_df
# --- end ----------------------------------------------------------------------


In [ ]:
bundles= []

# for ticker in tickers:
#     print(f'scraping: {ticker}')
#     try:
#         stock = finvizfinance(ticker)
#         stock.ticker_charts(out_dir='asset')
#     except Exception as e:
#         print(f'error scraping {ticker}: {e}')
#         continue

#     stock_fundamentals = stock.ticker_fundament()
#     stock_description = stock.ticker_description()
#     outer_ratings_df = stock.ticker_outer_ratings()
#     news_df = stock.ticker_news()
#     inside_trader_df = stock.ticker_inside_trader()

#     stock_charts = Statements()
#     income_df_q= stock_charts.get_statements(ticker=ticker, statement='I', timeframe='Q')
#     cashflow_df_q= stock_charts.get_statements(ticker=ticker, statement='C', timeframe='Q')
#     balance_df_q= stock_charts.get_statements(ticker=ticker, statement='B', timeframe='Q')

#     bundle= {
#         'fund': stock_fundamentals,
#         'inc': income_df_q,
#         'cf': cashflow_df_q,
#         'bs': balance_df_q
#     }
#     bundles.append(bundle)

for ticker in tickers:
    print(f'scraping: {ticker}')
    try:
        stock = yf.Ticker(ticker)
    except Exception as e:
        print(f'error scraping {ticker}: {e}')
        continue

    stock_fundamentals = stock.info
    
    income_df_q= stock.quarterly_incomestmt
    cashflow_df_q= stock.quarterly_cashflow
    balance_df_q= stock.quarterly_balance_sheet

    bundle= {
        'fund': stock_fundamentals,
        'inc': income_df_q,
        'cf': cashflow_df_q,
        'bs': balance_df_q
    }
    bundles.append(bundle)




scraping: EQIX
scraping: DLR
scraping: PWR
scraping: ETN


Exception ignored from cffi callback <function buffer_callback at 0x29d7b8cc0>:
Traceback (most recent call last):
  File "/Users/rssantanu/anaconda3/envs/gene_expression_312/lib/python3.12/site-packages/curl_cffi/curl.py", line 100, in buffer_callback
    @ffi.def_extern()
    
KeyboardInterrupt: 


scraping: ABB
scraping: VRT
scraping: TT
scraping: JCI


In [ ]:
rankings, ticker_df = rank_stocks(bundles)

In [ ]:
rankings

[('NVIDIA Corporation', 86.7, 'BUY (High Conviction)'),
 ('SK hynix Inc.', 80.2, 'BUY (High Conviction)'),
 ('Arista Networks Inc', 78.2, 'BUY (High Conviction)'),
 ('Micron Technology, Inc.', 70.7, 'ACCUMULATE / HOLD'),
 ('Taiwan Semiconductor Manufacturing Company Limited',
  67.6,
  'ACCUMULATE / HOLD'),
 ('Waaree Energies Limited', 60.1, 'ACCUMULATE / HOLD'),
 ('Analog Devices, Inc.', 59.5, 'AVOID / WATCHLIST'),
 ('Adobe Inc.', 58.7, 'AVOID / WATCHLIST'),
 ('Novo Nordisk A/S', 57.1, 'AVOID / WATCHLIST'),
 ('Lam Research Corporation', 55.2, 'AVOID / WATCHLIST'),
 ('KLA Corporation', 54.3, 'AVOID / WATCHLIST'),
 ('Seagate Technology Holdings plc', 50.1, 'AVOID / WATCHLIST'),
 ('Marvell Technology, Inc.', 49.8, 'AVOID / WATCHLIST'),
 ('Alphabet Inc.', 48.8, 'AVOID / WATCHLIST'),
 ('Broadcom Inc.', 48.5, 'AVOID / WATCHLIST'),
 ('Vertiv Holdings Co', 46.4, 'AVOID / WATCHLIST'),
 ('Texas Instruments Incorporated', 46.0, 'AVOID / WATCHLIST'),
 ('Cadence Design Systems, Inc.', 44.1, 'AVOID

In [ ]:
ticker_df.sort_values(by='Score', ascending=False)

,Company,Gross Margin,Oper. Margin,ROE,Forward P/E,PEG,Revenue TTM Growth,FCF Margin,Debt/Equity,Current Ratio,Growth,Profitability,Valuation,Safety,Score
37,NVIDIA Corporation,69.847000,60.842997,109.417000,45.208736,1.0002,55.6,41.662778,0.105841,4.214000,1.000000,0.998725,0.499950,0.947079,86.666690
40,SK hynix Inc.,53.907000,41.439998,39.278000,8.293203,0.7065,35.4,18.589259,0.279697,1.784965,0.868571,0.693452,1.000000,0.556393,80.206601
11,Arista Networks Inc,64.236000,44.730000,33.638000,15.808840,2.4023,30.4,47.542648,0.000000,3.333045,0.725714,0.896700,0.581410,1.000000,78.207679
18,"Micron Technology, Inc.",39.791000,32.638000,17.198999,17.017872,0.1752,46.0,0.000000,0.318062,2.754711,1.000000,0.206975,0.910395,0.779647,70.663833
23,Taiwan Semiconductor Manufacturing Company Lim...,58.976000,50.578000,34.657001,36.504950,1.5175,30.3,26.993854,0.189980,2.370496,0.722857,0.796890,0.433037,0.747634,67.632852
47,Waaree Energies Limited,32.509997,19.233999,33.703000,23.280167,0.0000,69.7,1.419375,0.157436,1.497452,1.000000,0.232808,0.597137,0.545645,60.097344
21,"Analog Devices, Inc.",60.151000,28.551000,5.652000,31.566313,0.8240,24.6,35.062118,0.254904,2.324431,0.560000,0.530850,0.650602,0.703656,59.545379
39,Adobe Inc.,89.138000,36.289000,52.875000,17.202919,1.0033,10.7,38.769730,0.563806,1.018725,0.162857,0.927408,0.906266,0.222778,58.706279
42,Novo Nordisk A/S,84.258000,43.521000,79.168000,13.012284,1.4816,12.9,19.236810,0.590649,0.784037,0.225714,0.856315,0.861523,0.204676,57.069102
36,Lam Research Corporation,49.313000,34.354000,62.261003,35.522247,2.3915,27.7,31.727210,0.454743,2.210118,0.648571,0.711618,0.232085,0.575158,55.235185


In [3]:
stock= yf.Ticker("NVDA")

In [5]:
(stock.info)

{'address1': '2788 San Tomas Expressway',
 'city': 'Santa Clara',
 'state': 'CA',
 'zip': '95051',
 'country': 'United States',
 'phone': '408 486 2000',
 'website': 'https://www.nvidia.com',
 'industry': 'Semiconductors',
 'industryKey': 'semiconductors',
 'industryDisp': 'Semiconductors',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': "NVIDIA Corporation, a computing infrastructure company, provides graphics and compute and networking solutions in the United States, Singapore, Taiwan, China, Hong Kong, and internationally. The Compute & Networking segment includes its Data Centre accelerated computing platforms and artificial intelligence solutions and software; networking; automotive platforms and autonomous and electric vehicle solutions; Jetson for robotics and other embedded platforms; and DGX Cloud computing services. The Graphics segment offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and 

In [ ]:
stock.balancesheet

,2025-01-31,2024-01-31,2023-01-31,2022-01-31,2021-01-31
Free Cash Flow,6.085300e+10,2.702100e+10,3.808000e+09,8.132000e+09,NaN
Repurchase Of Capital Stock,-3.370600e+10,-9.533000e+09,-1.003900e+10,0.000000e+00,NaN
Repayment Of Debt,-1.250000e+09,-1.250000e+09,0.000000e+00,-1.000000e+09,NaN
Issuance Of Debt,NaN,0.000000e+00,0.000000e+00,4.977000e+09,4.968000e+09
Capital Expenditure,-3.236000e+09,-1.069000e+09,-1.833000e+09,-9.760000e+08,NaN
Interest Paid Supplemental Data,2.460000e+08,2.520000e+08,2.540000e+08,2.460000e+08,NaN
Income Tax Paid Supplemental Data,1.511800e+10,6.549000e+09,1.404000e+09,3.960000e+08,NaN
End Cash Position,8.589000e+09,7.280000e+09,3.389000e+09,1.990000e+09,NaN
Beginning Cash Position,7.280000e+09,3.389000e+09,1.990000e+09,8.470000e+08,NaN
Changes In Cash,1.309000e+09,3.891000e+09,1.399000e+09,1.143000e+09,NaN
